# Simulator
---
Dieses Modul ist dafür zuständig, verschiedene Aspekte des Stromverbrauchs, der Stromerzeugung und der Stromkosten basierend auf unterschiedlichen Benutzereingaben zu simulieren. Es hilft bei der Analyse des Stromverbrauchs, der Erzeugung und der Kosten für verschiedene Benutzertypen wie Verbraucher, Prosumer und Flexumer.

#### Funktionsweise

In diesem Modul werden die folgenden Funktionen verwendet:

1. `load_data()`: Diese Funktion lädt Daten aus einer Excel-Datei und überprüft, ob die Datei existiert. Falls ja, werden relevante Spalten eingelesen und zurückgegeben.


2. `simulate_power_consumption(data, annual_consumption, tariff_input)`: Simuliert den Stromverbrauch basierend auf dem jährlichen Verbrauch und dem SLP-Basisprozentwert. Eine zufällige Variation wird hinzugefügt, um die Verbrauchswerte realitätsnäher zu gestalten.


3. `simulate_power_production(data, pv_capacity`: Simuliert die Stromerzeugung basierend auf der PV-Kapazität (Photovoltaik). Je nach Kapazität wird die Erzeugung angepasst und eine zufällige Variation hinzugefügtsimulate_power_production(data, pv_capacity): Simuliert die Stromerzeugung basierend auf der PV-Kapazität (Photovoltaik). Je nach Kapazität wird die Erzeugung angepasst und eine zufällige Variation hinzugefügt`.


4. `simulate_power_price(data, annual_consumption, current_tariff, current_base_price)`: Simuliert die Strompreise basierend auf dem jährlichen Verbrauch, dem aktuellen Tarif und dem Grundpreis. Es berechnet feste und variable Stromtarife sowie die Marktstrompreise.


5. `simulate_grid_consumption(data, user_type`: Simuliert den Netzverbrauch basierend auf dem Benutzertyp (Verbraucher, Prosumer, Flexumer). Je nach Typ wird der Netzverbrauch entsprechend angepasst.


6. `simulate_power_production_usage(data`: Simuliert die Nutzung der Stromerzeugung basierend auf dem Verbrauch. Diese Funktion stellt sicher, dass die Erzeugung so weit wie möglich genutzt wird, bevor Netzstrom verwendet wird.


7. `simulate_battery_usage(data, battery_capacity`: Simuliert die Nutzung eines Stromspeichers (Batterie) basierend auf der Batteriekapazität. Es wird berücksichtigt, ob überschüssige Produktion gespeichert oder Verbrauch aus der Batterie gedeckt werden kann.


8. `run_simulator(annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity)`: Dies ist die Hauptfunktion, die die Simulation basierend auf den Benutzereingaben ausführt. Sie kombiniert alle oben genannten Funktionen, um den gesamten Prozess zu simulieren, einschließlich Verbrauch, Erzeugung, Batterieeinsatz und Strompreise.

#### Quellcode

In [2]:
import os
import pandas as pd
import random
import numpy as np


df = None
data = None

def load_data():
    global df
    excel_file_path = 'SGS/Output-Data/ETL-Prozess_Ergebnisse.xlsx'
    
    # Überprüfen, ob die Excel-Datei existiert
    if os.path.exists(excel_file_path):
        # Lesen der relevanten Spalten aus der Excel-Datei
        df = pd.read_excel(excel_file_path, usecols=['Zeitstempel', 'SLP-Basisprozentwert', 'PV-1kWp', 'Strommarktpreis [€/kWh]'], nrows=35040)
        data = df[['Zeitstempel', 'SLP-Basisprozentwert']].copy()
        return data
    else:
        print(f"Error: Datei konnte nicht gefunden werden {excel_file_path}")
        return None, None

# Simuliert den Stromverbrauch basierend auf dem jährlichen Verbrauch und dem SLP-Basisprozentwert
def simulate_power_consumption(data, annual_consumption, tariff_input):
    try:
        annual_consumption = float(annual_consumption)
        # Berechnen des Stromverbrauchs basierend auf dem jährlichen Verbrauch und dem SLP-Basisprozentwert
        power_consumption = data['SLP-Basisprozentwert'] * annual_consumption / 100
        variation = random.uniform(0.001, 0.009)
        power_consumption *= (1 + variation)
        data['Stromverbrauch [kWh]'] = power_consumption
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation des Stromverbrauchs: {e}")
        return False

# Simuliert die Stromerzeugung basierend auf der PV-Kapazität
def simulate_power_production(data, pv_capacity):
    try:  
        if pv_capacity == 'PV-1kWp':
            # Anpassen der PV-Erzeugungswerte um eine zufällige Variation
            adjusted_pv_1kw = df['PV-1kWp'] * np.random.uniform(0.999, 1.001)
            power_production = adjusted_pv_1kw
            data['Stromerzeugung [kWh]'] = power_production
        elif pv_capacity in ['PV-5kWp', 'PV-10kWp', 'PV-15kWp', 'PV-20kWp', 'PV-25kWp', 'PV-30kWp']:
            # Berechnen der PV-Erzeugung für verschiedene Kapazitäten
            adjusted_pv_1kw = df['PV-1kWp'] * np.random.uniform(0.999, 1.001)
            scaling_factor = int(pv_capacity.split('-')[1].replace('kWp', ''))
            power_production = adjusted_pv_1kw * scaling_factor
            data['Stromerzeugung [kWh]'] = power_production
        
        return simulate_power_production_usage(data)
    except Exception as e:
        print(f"Fehler bei der Simulation der Stromerzeugung: {e}")
        return False

# Simuliert die Strompreise basierend auf dem jährlichen Verbrauch, dem aktuellen Tarif und dem Grundpreis
def simulate_power_price(data, annual_consumption, current_tariff, current_base_price):
    global df 
    annual_consumption = float(annual_consumption)
    current_tariff = float(current_tariff)
    current_base_price = float(current_base_price)
        
    # Berechnen der festen und variablen Stromtarife
    fixed_tariff_values = (data['Lasten Netz [kWh]'] * (current_tariff/100)) + ((current_base_price * 12) / 35040)
    variable_tariff_values = (data['Lasten Netz [kWh]'] * df['Strommarktpreis [€/kWh]']) + ((current_base_price * 12) / 35040)
        
    data['Fixer Stromtarif [€/kWh]'] = fixed_tariff_values
    data['Dynamischer Stromtarif [€/kWh]'] = variable_tariff_values
    data['Strommarktpreis [€/kWh]'] = df['Strommarktpreis [€/kWh]']
    return True 

# Simuliert den Netzverbrauch basierend auf dem Benutzertyp (Verbraucher, Prosumer, Flexumer)
def simulate_grid_consumption(data, user_type):
    # Initialisieren der Netzlast mit 0
    data['Lasten Netz [kWh]'] = 0.0
    
    if user_type == 'consumer':
        # Verbraucher verbrauchen direkt aus dem Netz
        data['Lasten Netz [kWh]'] = data['Stromverbrauch [kWh]']
    else:
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            
            if user_type == 'prosumer':
                # Prosumer verbrauchen zuerst Eigenproduktion, dann Netzstrom
                lasten = consumption - data.at[index, 'Lasten Stromerzeugung [kWh]']
                data.at[index,'Lasten Netz [kWh]'] =  lasten if lasten > 0 else 0.0
            if user_type == 'flexumer':
                # Flexumer nutzen zuerst Eigenproduktion und Batterie, dann Netzstrom
                lasten = consumption - data.at[index, 'Lasten Stromerzeugung [kWh]'] - data.at[index, 'Lasten Stromspeicher [kWh]']
                data.at[index,'Lasten Netz [kWh]'] = lasten if lasten > 0 else 0.0
    return True

# Simuliert die Nutzung der Stromerzeugung basierend auf dem Verbrauch
def simulate_power_production_usage(data):
    try:
        # Initialisieren der Lasten Stromerzeugung mit 0
        data['Lasten Stromerzeugung [kWh]'] = 0.0
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            if production == 0:
                data.at[index, 'Lasten Stromerzeugung [kWh]'] = 0.0
            else:
                # Nutzung der Produktion basierend auf dem Verbrauch
                lasten = consumption if (production-consumption) >= 0 else production
                data.at[index, 'Lasten Stromerzeugung [kWh]'] = lasten
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation Lasten Stromerzeugung: {e}")
        return False

# Simuliert die Nutzung eines Stromspeichers (Batterie) basierend auf der Batteriekapazität
def simulate_battery_usage(data, battery_capacity):
    try:
        capacity_mapping = {
            '2,5 kWh': 2.5,
            '5,1 kWh': 5.1,
            '7,7 kWh': 7.7,
            '10,2 kWh': 10.2,
            '12,8 kWh': 12.8
        }

        batt_capacity = capacity_mapping.get(battery_capacity, 0.0)

        battery_capacity = float(batt_capacity)

        data['Stromspeicher [kWh]'] = 0.0
        data['Lasten Stromspeicher [kWh]'] = 0.0
        
        batt_15mins_peak = batt_capacity / 4
     
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            if production == 0:
                # Verbrauch aus der Batterie wenn keine Produktion stattfindet
                batt_consumption = consumption if consumption <= batt_15mins_peak else batt_15mins_peak
                if batt_consumption <= battery_capacity:
                    data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption 
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = consumption
                    battery_capacity -= batt_consumption
                else:
                    data.at[index, 'Stromspeicher [kWh]'] = 0.0
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = battery_capacity
                    battery_capacity = 0.0
            else:
                if (production - consumption) > 0:
                    # Speichern überschüssiger Produktion in der Batterie
                    production_rest = batt_15mins_peak if (production - consumption) >= batt_15mins_peak else (production - consumption)
                    if (data.at[index - 1, 'Stromspeicher [kWh]'] + production_rest) <= batt_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity + production_rest
                        battery_capacity += production_rest
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = batt_capacity 
                        battery_capacity = batt_capacity
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = 0.0
                else:
                    # Verbrauch aus der Batterie bei geringer Produktion
                    batt_consumption = (consumption - production) if (consumption - production) <= batt_15mins_peak else batt_15mins_peak
                    if batt_consumption <= battery_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption
                        data.at[index, 'Lasten Stromspeicher [kWh]'] = batt_consumption
                        battery_capacity -= batt_consumption
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = 0.0
                        data.at[index, 'Lasten Stromspeicher [kWh]'] = battery_capacity
                        battery_capacity = 0.0
                        
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation der Batterienutzung: {e}")
        return False

# Hauptfunktion zum Ausführen der Simulation basierend auf den Benutzereingaben
def run_simulator(annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity):
    # Simulation des Stromverbrauchs basierend auf dem jährlichen Verbrauch und dem Tarif
    simulate_power_consumption(data, annual_input, tariff_input)
    if user_type != 'consumer':
        # Simulation der Stromerzeugung bei Prosumer und Flexumer
        simulate_power_production(data, pv_capacity)
    if user_type == 'flexumer':
        # Simulation der Batterienutzung bei Flexumer
        simulate_battery_usage(data, battery_capacity)
    # Simulation des Netzverbrauchs
    simulate_grid_consumption(data, user_type)
    # Simulation der Strompreise
    simulate_power_price(data, annual_input, tariff_input, base_input)